In [12]:
from datetime import datetime

import backtrader as bt
import matplotlib.pyplot as plt
import akshare as ak
import pandas as pd

In [13]:
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
# 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
stock_hfq_df = ak.stock_zh_a_hist(symbol="000001", adjust="hfq").iloc[:, :6]

# 处理字段命名，以符合 Backtrader 的要求
# 日期	开盘	收盘	最高	最低	成交量
stock_hfq_df.columns = [
  'date',
  'open',
  'close',
  'high',
  'low',
  'volumn'
]
# # 把 date 作为日期索引，以符合 Backtrader 的要求
stock_hfq_df.index = pd.to_datetime(stock_hfq_df['date'])


In [ ]:
class MyStrategy(bt.Strategy):
  """
  主策略程序
  """
  params = (('maperiod', 20),)  # 全局设定交易策略的参数

  def __init__(self):
    """
    初始化函数
    """
    self.data_close = self.datas[0].close # 指定价格序列
    # 初始化交易指令、买卖价格和手续费
    self.order = None
    self.buy_price = None
    self.buy_comm = None
    # 添加移动均线指标
    self.sma = bt.indicators.SimpleMovingAverage(
      self.datas[0], period=self.params.maperiod
    )

  def next(self):
    """
    执行逻辑
    """
    if self.order:  # 检查是否有指令等待执行
      return
    # 检查是否持仓
    if not self.position: # 没有持仓
      if self.data_close[0] > self.sma[0]:  # 执行买入条件判断：收盘价格上涨突破20日均线
        self.order = self.buy(size=100) # 执行买入
      else:
        if self.data_close[0] < self.sma[0]:   # 执行卖出条件判断：收盘价格跌破20日均线
          self.order = self.sell(size=100)

In [ ]:
cerebro = bt.Cerebro()
start_time = datetime(1991, 4, 3)
